Course: Language as Data, University of Göttingen

# Week 6: N-Gram Language Models

In this lab, we learn how to build an n-gram based language model using the book [Emma](https://www.gutenberg.org/cache/epub/158/pg158.txt) as training data. The book was written by Jane Austen in the early 19th century and is made available by the Gutenberg project.

In [1]:
!pip install -U pip
!pip install -U nltk==3.4

  Using cached pip-24.3.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-24.3.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2
  Using cached nltk-3.4.zip (1.4 MB)
  Preparing metadata (setup.py) ... done
  Using cached singledispatch-4.1.0-py2.py3-none-any.whl.metadata (3.8 kB)
Using cached singledispatch-4.1.0-py2.py3-none-any.whl (6.7 kB)
  Created wheel for nltk: filename=nltk-3.4-py3-none-any.whl size=1436383 sha256=87062b1c6e3eef71fb3a437cae4032ba1ea032bfb5e46088b1ab9790e3162b14
  Stored in directory: /home/jopi-lima/.cache/pip/wheels/db/96/da/0a26fbd3f96b179cc14b813434a0c324a08c0684afdd524c73
Successfully built nltk


In [1]:
import os
import requests
import io 
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jopi-lima/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
with open("jane_austen_emma.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of characters:", len(raw_text))
print(raw_text[:500])

Total number of characters: 879603

VOLUME I

CHAPTER I


Emma Woodhouse, handsome, clever, and rich, with a comfortable home and
happy disposition, seemed to unite some of the best blessings of
existence; and had lived nearly twenty-one years in the world with very
little to distress or vex her.

She was the youngest of the two daughters of a most affectionate,
indulgent father; and had, in consequence of her sister’s marriage,
been mistress of his house from a very early period. Her mother had
died too long ago for her to have 


## 1. Pre-processing
For your project, you will use the tokenizer you trained in the previous project. To keep the illustration simple, we use existing tokenizers here. 

In [3]:
from nltk import word_tokenize, sent_tokenize 
tokenized_text = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in sent_tokenize(raw_text)]
print(tokenized_text[0:5])


[['volume', 'i', 'chapter', 'i', 'emma', 'woodhouse', ',', 'handsome', ',', 'clever', ',', 'and', 'rich', ',', 'with', 'a', 'comfortable', 'home', 'and', 'happy', 'disposition', ',', 'seemed', 'to', 'unite', 'some', 'of', 'the', 'best', 'blessings', 'of', 'existence', ';', 'and', 'had', 'lived', 'nearly', 'twenty-one', 'years', 'in', 'the', 'world', 'with', 'very', 'little', 'to', 'distress', 'or', 'vex', 'her', '.'], ['she', 'was', 'the', 'youngest', 'of', 'the', 'two', 'daughters', 'of', 'a', 'most', 'affectionate', ',', 'indulgent', 'father', ';', 'and', 'had', ',', 'in', 'consequence', 'of', 'her', 'sister', '’', 's', 'marriage', ',', 'been', 'mistress', 'of', 'his', 'house', 'from', 'a', 'very', 'early', 'period', '.'], ['her', 'mother', 'had', 'died', 'too', 'long', 'ago', 'for', 'her', 'to', 'have', 'more', 'than', 'an', 'indistinct', 'remembrance', 'of', 'her', 'caresses', ';', 'and', 'her', 'place', 'had', 'been', 'supplied', 'by', 'an', 'excellent', 'woman', 'as', 'governess'

Then we preprocess the text. We use the *padded_everygram_pipeline* function from nltk. It pads the sentence and then makes so-called *everygrams*. Let's see what this looks like. **Make sure you understand what is meant by everygrams here. How does the padding change when you change n? How dow the n-grams change if you change n?**

In [5]:
from nltk.lm.preprocessing import padded_everygram_pipeline
# Set the n-gram size
n = 3

# Check pre-processing on a subset: 
ngram_data, padded = padded_everygram_pipeline(n, tokenized_text[0:10])

# What is the effect of padding? 
print("PADDING:")
print(list(padded))

# Which ngrams do we get? 
print("\n\nNGRAMS:")
for ngrams in ngram_data: 
    print(list(ngrams))
    print()

PADDING:
['<s>', '<s>', 'volume', 'i', 'chapter', 'i', 'emma', 'woodhouse', ',', 'handsome', ',', 'clever', ',', 'and', 'rich', ',', 'with', 'a', 'comfortable', 'home', 'and', 'happy', 'disposition', ',', 'seemed', 'to', 'unite', 'some', 'of', 'the', 'best', 'blessings', 'of', 'existence', ';', 'and', 'had', 'lived', 'nearly', 'twenty-one', 'years', 'in', 'the', 'world', 'with', 'very', 'little', 'to', 'distress', 'or', 'vex', 'her', '.', '</s>', '</s>', '<s>', '<s>', 'she', 'was', 'the', 'youngest', 'of', 'the', 'two', 'daughters', 'of', 'a', 'most', 'affectionate', ',', 'indulgent', 'father', ';', 'and', 'had', ',', 'in', 'consequence', 'of', 'her', 'sister', '’', 's', 'marriage', ',', 'been', 'mistress', 'of', 'his', 'house', 'from', 'a', 'very', 'early', 'period', '.', '</s>', '</s>', '<s>', '<s>', 'her', 'mother', 'had', 'died', 'too', 'long', 'ago', 'for', 'her', 'to', 'have', 'more', 'than', 'an', 'indistinct', 'remembrance', 'of', 'her', 'caresses', ';', 'and', 'her', 'place', 

## 2. Training the N-gram Model

We can now train a model on the pre-processed data. We use a Maximum Likelihood Estimator (MLE) using trigrams as the maximum n-gram size. 


In [6]:
from nltk.lm import MLE

n=3

# Train data is an iterator over the pre-processed input
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)

model = MLE(n)
model.fit(train_data, padded_sents)



In [7]:
# Let's have a look at the vocabulary
print(model.vocab)
print()
print(model.vocab.counts.most_common(50))
print()
print(model.vocab.counts.most_common()[-50:])

# You can see that the text is much cleaner than the corpora that were crawled from the web

<Vocabulary with cutoff=1 unk_label='<UNK>' and 10144 items>

[(',', 12018), ('<s>', 11700), ('</s>', 11700), ('to', 5137), ('.', 5128), ('the', 5120), ('and', 4541), ('of', 4258), ('a', 3060), ('i', 2919), ('was', 2373), ('her', 2367), (';', 2353), ('it', 2338), ('not', 2242), ('she', 2219), ('in', 2138), ('“', 2099), ('”', 2090), ('be', 1958), ('you', 1876), ('that', 1754), ('he', 1734), ('had', 1611), ('as', 1429), ('have', 1312), ('for', 1294), ('is', 1221), ('but', 1207), ('with', 1202), ('very', 1173), ('his', 1126), ('’', 1114), ('mr.', 1089), ('!', 1063), ('at', 1016), ('so', 929), ('s', 914), ('could', 825), ('all', 820), ('emma', 819), ('would', 809), ('been', 753), ('my', 719), ('him', 706), ('no', 684), ('on', 680), ('mrs.', 669), ('any', 652), ('?', 621)]

[('vouch', 1), ('knightley—or', 1), ('churchills—or', 1), ('unbleached', 1), ('nobility', 1), ('liberally', 1), ('created', 1), ('—or', 1), ('regretted.—the', 1), ('gradual', 1), ('impair.—perhaps', 1), ('clergyman', 1),

# 3. Using the N-gram Language Model
An ngram-based language model simply counts the frequency of n-grams during training and stores them in an easily accessible counter representation. 

In [8]:
print(model.counts)

# counts for unigrams:
print(model.counts['not']) # i.e. Count('not')
# count for bigrams
print(model.counts[['was']]['not']) # i.e. Count('not'|'was')
# count for trigrams
print(model.counts[['emma', 'was']]['not']) # i.e. Count('not'|'emma was')

<NgramCounter with 3 ngram orders and 612441 ngrams>
2242
161
5


Based on the ngram-frequencies, it calculates the probability of a word in a certain context. 
For the MLE model, the probability is calculated as the relative frequency. **Note that the model score is lower than the probability.** This has to do with the padding tokens. **Make sure you understand how the score is adjusted.**


In [9]:
all_tokens = [tok for sent in tokenized_text for tok in sent]
num_tokens = len(all_tokens)
num_sentences = len(tokenized_text)

model_score = model.score('not')
probability = model.counts['not']/num_tokens
                                    

print("\nProbability of the word 'not'")
print("{:.5f}".format(model_score))
print("{:.5f}".format(probability))

print("\nAdjust for padding tokens")
all_padding_tokens = num_sentences * (n-1) * 2
print(num_tokens, all_padding_tokens)

adjusted_probability = model.counts['not']/(num_tokens + all_padding_tokens)
print("{:.5f}".format(adjusted_probability))

print("\nProbabilities padding tokens")
print("{:.5f}".format(model.score('<s>')))
print("{:.5f}".format(model.score('</s>')))



Probability of the word 'not'
0.01068
0.01202

Adjust for padding tokens
186597 23400
0.01068

Probabilities padding tokens
0.05572
0.05572


In [10]:
# We can also calculate the frequency for higher n-grams.
# bigram
print(model.score('not', ['was']))  # P('not'|'is')
# trigram
print(model.score('not', ['emma', 'was']))  # P('not'|'emma is')
print()


0.06784660766961652
0.078125



In [11]:
# To avoid underflow when working with many small score values, we usually work with log probabilities instead. 
# This can be done with the `logscore` method.
print(model.score('sure', ['very']))
print(model.logscore('sure', ['very']))



0.010230179028132993
-6.611024797307352


## 4. The Unknown Word Problem

In [12]:
# The vocabulary helps us handle words that have not occurred during training.
# If we lookup the vocab on unseen sentences not from the training data, 
# it automatically replace words not in the vocabulary with `<UNK>`.
print(model.vocab.lookup('emma did not feel challenged at all.'.split()))

('emma', 'did', 'not', 'feel', '<UNK>', 'at', 'all.')


The standard MLE language model has a problem with unknown words. 

In [13]:
# Items that are not seen during training are mapped to the vocabulary's "unknown label" token.  This is "<UNK>" by default.
print(model.score("<UNK>") == model.score("challenged"))

# The MLE model does not apply any smoothing, so the probability for UNK is 0
print(model.score("<UNK>"),model.logscore("<UNK>") )

# As a consequence, the probability for a phrase containing an unknown word is also 0. 
print(model.score('challenged', ['not', 'feel']), model.logscore('challenged', ['not', 'feel']))

True
0.0 -inf
0.0 -inf


This problem can be solved with more advanced smoothing techniques, for example Laplace smoothing.

In [14]:
from nltk.lm import Laplace
n = 3
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)
smoothed_model_small =  Laplace(n)
smoothed_model_small.fit(train_data, padded_sents)
print(smoothed_model_small.score('not'))
print(smoothed_model_small.score('feel'))
print(smoothed_model_small.score('challenged', ['not', 'feel']))
print()
print(smoothed_model_small.logscore('not'))
print(smoothed_model_small.logscore('feel'))
print(smoothed_model_small.logscore('challenged', ['not', 'feel']))

0.01018892437119846
0.00043608414607001876
9.850275807722617e-05

-6.616854433290205
-11.163105837655321
-13.309476353841106


# 5. Sampling

In [15]:
smoothed_model_small.generate(text_seed=["I", "did", "not", "feel"], num_words=3, random_seed=1)

['afraid', '.', '</s>']